# **위도 / 경도 정보 가져오기**
- **[KaKao API 가져오기](https://medium.com/@john_analyst/%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%9D%84-%ED%99%9C%EC%9A%A9%ED%95%9C-%EC%B9%B4%EC%B9%B4%EC%98%A4-api%EB%A1%9C-%EC%9C%84%EA%B2%BD%EB%8F%84-%EA%B5%AC%ED%95%98%EA%B8%B0-69bc51697753)**
- **[GeoCoder 를 활용한 변환](https://m.blog.naver.com/PostView.nhn?blogId=kcscpr&logNo=220903590992&proxyReferer=https:%2F%2Fwww.google.com%2F)**
- **[GeoCoder Github](https://github.com/yjw5344/Geocoder-Python)**
- **[Juso API 게시판에 올라온 GET 방식](https://juso.go.kr/addrlink/devCenterEventBoardDetail.do?regSn=79&noticeType=T)**
- **[공공 데이터 DB 자료를 활용한 방식](https://stricky.tistory.com/116)**
- **[Naver, Daum API 활용하기](https://mentha2.tistory.com/176)**

## **1 Source 의 데이터 정보를 결과값 확인하기**
Validation Check Function

In [1]:
from glob import glob
file_excel = sorted(glob("./data/store*.xlsx"))
file_csv = sorted(glob("./data/store.csv"))
print(file_csv, file_excel)

import pandas as pd
data_excel = pd.read_excel(file_excel[0])
data_csv = pd.read_csv(file_csv[0], header=None)
data_csv.columns = ['master','artist','title','name','phone','lat','lon','address','addressraw','type']
print(data_csv.shape, data_excel.shape)
data_excel.head(2)

['./data/store.csv'] ['./data/store.xlsx']
(150, 10) (150, 11)


,연번,상호명,점주명,연락처,주소,업종,행정동,사진,임대차,비고,Unnamed: 10
0,1,모든홍삼,주옥자,010-7908-0324,"은천로 28, 봉일프라자 나-120",기타판매,은천동,X,X,개별점포,NaN
1,2,종로떡집,김기철,010-4606-7280,국회단지길 21,떡집,은천동,X,X,개별점포,NaN


## **2 Building the Source Token**
Checking 을 위한 Token Dict 만들기

In [2]:
# for _ in range(len(data_excel)):
token_source = {}
for _ in range(len(data_excel)):
    title = "".join(data_excel.loc[_, ["상호명"]].values)
    token_source[title] = {}
    name = "".join(data_excel.loc[_, ["점주명"]].values)
    phone = "".join(data_excel.loc[_, ["연락처"]].values)
    address = "".join(data_excel.loc[_, ["주소"]].values)
    types = "".join(data_excel.loc[_, ["업종"]].values)
    token_source[title]["name"] = name
    token_source[title]["phone"] = phone
    token_source[title]["address"] = address
    token_source[title]["types"] = types

len(token_source)

150

## **3 Adding the Data**
Checking Token 을 활용한 Table Validation Check

In [3]:
data_csv.head(2)

,master,artist,title,name,phone,lat,lon,address,addressraw,type
0,1,1,카페비온,주옥자,010-7908-0324,37.462847,126.917727,서울특별시 관악구 난향길 24 현경빌딩,난향길 24,카페
1,1,1,김현애보습학원,김기철,010-4606-7280,37.462919,126.918079,서울특별시 관악구 난향길 14,"난향길 14, 2층",학원


In [4]:
name, phone, address, types = [], [], [], []
for _ in range(len(data_csv)):
    title = "".join(data_csv.loc[_,'title'])
    name.append(token_source[title]["name"])
    phone.append(token_source[title]["phone"])
    address.append(token_source[title]["address"])
    types.append(token_source[title]["types"])
data_csv['name'] = name
data_csv['phone'] = phone
# data_csv.insert(8, '주소', address)
# data_csv.insert(10, '업종', types)
data_csv.to_csv("./data/store_fin.csv", index=None)
data_csv.head(2)

,master,artist,title,name,phone,lat,lon,address,주소,addressraw,업종,type
0,1,1,카페비온,유민영,010-9688-7097,37.462847,126.917727,서울특별시 관악구 난향길 24 현경빌딩,난향길 24,난향길 24,카페,카페
1,1,1,김현애보습학원,김현애,010-4192-2239,37.462919,126.918079,서울특별시 관악구 난향길 14,"난향길 14, 2층","난향길 14, 2층",학원,학원


<br/>

# **Naver Map 을 활용한 위도경도 추출**

- [주소와 좌표 검색 API 사용하기](https://navermaps.github.io/maps.js.ncp/docs/tutorial-3-geocoder-geocoding.example.html)
- [GeoCode in Ncloud](https://apidocs.ncloud.com/ko/ai-naver/maps_geocoding/geocode/)
- [MapView APIKey - Non Paid]
```
Client ID : dzhrati97r
Client Secret : 9990BNYwSIY9paMzgwLZn5hjx4XTSaB0NjtuoIeQ
```

## **1 Naver 정보 추출함수**
- API 활용가능 범위 신청내용 확인 : 최초 신청시 GeoCode 내용 미포함으로 403 오류가 발생
- [Map Full Activated APIKey - paid possable]
```
Client ID : wstocuk035
Client Secret : Na6VgNttRRMbehOv6Uswfw8iYld2INbvZMvGBOHQ
```

In [5]:
import requests, json
def naver_address(address, latlon='127.1052133,37.3595316'):
    r"""Naver Cloud API 를 활용한 주소 상세정보 추출기
    :param address: 은천로 28, 봉일프라자 나-120
    :param latlon : 추출 기준이 되는 위도, 경도값
    :return: list 결과값 출력
    """
    # address  = ""
    url_head = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='
    url      = f"{url_head}{address}&coordinate={latlon}"
    headers  = {
        "X-NCP-APIGW-API-KEY-ID" : "wstocuk035",
        "X-NCP-APIGW-API-KEY" : "Na6VgNttRRMbehOv6Uswfw8iYld2INbvZMvGBOHQ",    
    }
    resp = requests.get(url, headers=headers)
    resp = json.loads(resp.text)
    if resp['status'] == 'OK': # 상태값이 유효하고
        if resp['meta']['totalCount'] >= 1: # 결과값이 1개 이상일 떄
            return_keys = ['roadAddress', 'jibunAddress','englishAddress','x', 'y']
            return [resp['addresses'][0][_]  for _ in return_keys]
        else:
            return resp
    else:
        return resp

In [6]:
naver_address("서울 관악구 장군봉1길 32")

['서울특별시 관악구 장군봉1길 32',
 '서울특별시 관악구 봉천동 1527-1',
 '32, Janggunbong 1-gil, Gwanak-gu, Seoul, Republic of Korea',
 '126.9417827',
 '37.4816691']

In [13]:
naver_address("24시 세탁")

{'status': 'OK',
 'meta': {'totalCount': 0, 'count': 0},
 'addresses': [],
 'errorMessage': ''}

<br />

# **Project 2 광진구 사회적경제 지도**
2020 11 02

## **1 Loading the Data**
주소자료 가져오기

In [ ]:
import pandas as pd
table = pd.read_csv("./data/socialmap_gj.csv").fillna('')
table.head(3)

## **2 위도 경도 데이터 추가하기**
Naver Cloud API 활용

In [ ]:
result_list = []
from tqdm import tqdm
for _ in tqdm(table['address'].to_list()):
    try:
        _ = naver_address(_)
    except:
#         print(">>" + _ + "<<")
        _ = [""]*4
    result_list.append(_)

table.insert(2,"lat", [_[-1] for _ in result_list])
table.insert(2,"lon", [_[-2] for _ in result_list])
table.insert(2,"addressroad", [_[1] for _ in result_list])
table.insert(2,"addressfull", [_[0] for _ in result_list])
table.to_csv("./data/socialmap_gj_geo.csv", index=None)
table.head(3)

<br />

# **Project 1 관악구 아트테리어 지도**
2020 10 21

## **1 위치정보 Table 에 추출한 정보 적용하기**
- Table Update

In [ ]:
# 데이터 CSV 불러오기
import pandas as pd
file_name = "./data/ourtown_store.csv"
table = pd.read_csv(file_name, dtype={"연번":str}).fillna('')
table['연번'] = [str(int(float(_)))  for _ in table['연번']]
table.head(3)

In [ ]:
# result_list = []
# from tqdm import tqdm
# for _ in tqdm(table['주소'].to_list()):
#     try:
#         _ = naver_address(_)
#     except:
#         print(_)
#     result_list.append(_)

# table.insert(5,"lat", [_[-1] for _ in result_list])
# table.insert(5,"lon", [_[-2] for _ in result_list])
# table.insert(5,"addressroad", [_[1] for _ in result_list])
# table.insert(5,"address", [_[0] for _ in result_list])
# table.to_csv("./data/ourtown_store_geo2.csv", index=None)
# table.head(3)

In [ ]:
table = pd.read_csv("./data/ourtown_store_geo2.csv")
table.head(3)

## **2 geoPy 를 활용한 위도/ 경도**
**[geopy document](https://pypi.org/project/geopy/)**

In [ ]:
# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent='gwanak_store')
# text_address = '남부순환로231길 11'
# location = geolocator.geocode(text_address)
# location.raw

## **2 주소 테이블 내용 추가하기**
- "latitude" : 위도
- "longitude" : 경도

In [ ]:
# address = table['주소'].to_list()
# address_base = [_.split(',')[0]  for _ in address]

# # generate Instance
# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent='gwanak_store')
# address_rest, latitude, longitude = [], [], []

# # adding the longitude, latitude
# from tqdm import tqdm
# from time import sleep
# for _ in tqdm(address_base):
#     location = geolocator.geocode(_)
#     latitude.append(location.raw['lat'])
#     longitude.append(location.raw['lon'])
#     sleep(.5)

In [ ]:
# for _ in address:
#     if len(_.split(','))>1:
#         address_rest.append("".join(_.split(',')[1:]).strip())
#     else: address_rest.append("")

# table.insert(5,'주소_head', address_base)
# table.insert(6,'주소_rest', address_rest)
# table.insert(7,'latitude', latitude)
# table.insert(8,'longitude', longitude)
# # table.to_csv('./data/ourtown_store_geo.csv', index=None)
# table.head(3)

import pandas as pd
df_address = pd.read_csv('./data/ourtown_store_geo.csv')
df_address.head(3)

# **지도 APP 위에 Marking 작업 진행하기**
Naver, Google 는 각자의 Cloud 서비스를 가입한 이후에만 사용이 가능하게 되어있는 문제가 있었습니다. 반면 Kakao 는 회원가입 만으로도 가능하여 이를 내용으로 진행을 합니다.
- **[Google Map Marking](https://simplehanlab.github.io/react/google-map-react/)**

## **1 KAKAO 가입 및 인증키 추출하기**
- **[Kakao Map marking](https://velog.io/@bearsjelly/React-kakao-%EC%A7%80%EB%8F%84-%EB%9D%84%EC%9A%B0%EA%B8%B0-2-%EC%95%B1%ED%82%A4%EB%A5%BC-%EC%9D%B4%EC%9A%A9%ED%95%B4-%EC%A7%80%EB%8F%84-%EB%9D%84%EC%9A%B0%EA%B8%B0)**
- **[React Kakao 지도 띄우기](https://velog.io/@bearsjelly/React-kakao-%EC%A7%80%EB%8F%84-%EB%9D%84%EC%9A%B0%EA%B8%B0-2-%EC%95%B1%ED%82%A4%EB%A5%BC-%EC%9D%B4%EC%9A%A9%ED%95%B4-%EC%A7%80%EB%8F%84-%EB%9D%84%EC%9A%B0%EA%B8%B0)**
- **[JavaScript Key 를 활용하여 MAP 구현하기](https://apis.map.kakao.com/web/guide/)**
- [React.js 에서 Kakao Map](https://gingerkang.tistory.com/65)
```
네이티브 앱 키	8cd75a36aae8250edab5ef6526a6a910
REST API 키	120cc9724a3d342cb5fb753d9dd65092
JavaScript 키	04f123664e335dd987570b9400dbafdc
Admin 키	ebc038d9655428ef995949136ee682b7
```

## **2 Naver Cloud 에서 Web MAP 활용하기**
- **[React 에서 네이버에서 지도 Marker 활용](https://velog.io/@gwak2837/React%EC%97%90-%EB%84%A4%EC%9D%B4%EB%B2%84-%EC%A7%80%EB%8F%84-%EB%B0%8F-%EB%A7%88%EC%BB%A4-%ED%91%9C%EC%8B%9C%ED%95%98%EA%B8%B0)**
- **[Naver Cloud MAP API](https://www.ncloud.com/product/applicationService/maps)**
```python
Client ID : dzhrati97r
Client Secret : 9990BNYwSIY9paMzgwLZn5hjx4XTSaB0NjtuoIeQ
```

## **3 지도 데이터 Json 으로 추출하기**
CSV file to Json ENV file
```
 'split' : dict like {'index' -> [index], 'columns' -> [columns],
          'data' -> [values]}
        - 'records' : list like [{column -> value}, ... , {column -> value}]
        - 'index' : dict like {index -> {column -> value}}
        - 'columns' : dict like {column -> {index -> value}}
        - 'values' : just the values array
        - 'table' :
```

In [ ]:
table = pd.read_csv("./data/ourtown_store_geo2.csv")
table.head(3)

In [ ]:
# 상호명, 
import pandas as pd
file_name = "./data/ourtown_store_geo2.csv"
column_name = ["상호명", "address", "lat","lon"]
df_table = pd.read_csv(file_name).loc[:, column_name]
df_table.columns = ["name", 'address', 'lat', 'lon']
df_table.head(2)

In [ ]:
import json
df_list = [df_table.iloc[_, :].to_list() for _ in range(len(df_table))]
df_table.to_json("mapinfo.json", force_ascii=False, orient="records")
print("output file is Done.")
# df_table.to_json(force_ascii=False, orient="table")

<br/>

# **Arterior 작가 작업구역 선정**
https://navermaps.github.io/maps.js.ncp/docs/tutorial-3-drawing-restore.example.html